# Plot lineage tree of a clone from Chernyshev et al. 2023 dataset, IML3695 post-vax library

The tree leaves are annotated with the sequence's chimeric status.

In [1]:
using Pkg; Pkg.activate("..")
include("../src/utils.jl")

  Activating project at `~/ben/chimera_detection/chimera_detection/notebooks`


run_filter_from_files (generic function with 1 method)

In [2]:
# INPUTS

# I can't share the igdiscover data publicly because it's restricted access
# But you can request to download the raw data here https://doi.org/10.17044/scilifelab.21518142
# And the IgDiscover instructions are included in the corresponding paper https://doi.org/10.1038/s41467-023-37972-1
# members.tsv file obtained with the command ```igdiscover clonotypes --mismatches 0.2 --members members.tsv.gz > clonotypes.tsv```

igdiscover_dir = "../../data/igdiscover22/IML369/IgM/IML3695_25cycle/"
clonotype_members_file = joinpath(igdiscover_dir, "final", "members.tsv.gz")
CHMMAIRRa_out_file = joinpath(igdiscover_dir, "final", "CHMMAIRRa_out.tsv.gz")

# OUTPUTS
trees_dir = "../../outputs/plots/trees/"

"../../outputs/plots/trees/"

In [3]:
members = CSV.read(clonotype_members_file, DataFrame, delim = "\t", select = [:sequence_id, :VDJ_nt, :clone_id, :cdr3, :V_SHM, :sequence_alignment, :germline_alignment]);
unique!(members, :VDJ_nt)
# output of CHMMAIRRa
CHMMAIRRA_out = CSV.read(CHMMAIRRa_out_file, DataFrame, delim = "\t");
CHMMAIRRA_out[!,"chimeric"] = CHMMAIRRA_out[:,"chimera_probability"] .> 0.95;

In [4]:
members = leftjoin(members, CHMMAIRRA_out, on = :sequence_id, makeunique = true);
# get number of chimeric sequences per lineage
chimeras_per_lineage = combine(groupby(members, :clone_id), :chimeric => function(x) return sum(x) end => :n_chimeric, nrow => :n_sequences);
chimeras_per_lineage[!,"percent_chimeric"] = chimeras_per_lineage[:,:n_chimeric] ./ chimeras_per_lineage[:,:n_sequences] * 100;
sort!(chimeras_per_lineage, :n_chimeric, rev = true);

In [5]:
# a small lineage with a low-ish chimeric rate probably makes for a good visualization
clones_of_interest = chimeras_per_lineage[(chimeras_per_lineage.percent_chimeric .> 0.5) .& (chimeras_per_lineage.percent_chimeric .< 4) .& (chimeras_per_lineage.n_sequences .< 200),:]
clone_ids = clones_of_interest.clone_id;
clones_of_interest

Row,clone_id,n_chimeric,n_sequences,percent_chimeric
,Int64,Int64,Int64,Float64
1,842,5,129,3.87597
2,1409,5,175,2.85714
3,4207,5,136,3.67647
4,442,3,116,2.58621
5,656,3,79,3.79747
6,1876,3,99,3.0303
7,2093,3,170,1.76471
8,15018,3,91,3.2967
9,17456,3,132,2.27273


In [6]:
# uncomment if you want to plot all of the clones
#for (i, clone_id) in enumerate(clone_ids[21:50])
#    if i % 10 == 0
#        println(i/length(clone_ids) * 100, "%")
#    end
clone_id = 19811
seqnames, sequences, chimeric = get_clone_id_sequences(members, clone_id);
newt = seqs2fasttree_tree(sequences, seqnames);

seqname2chimeric = Dict(zip(seqnames, chimeric));
seqname2color = Dict()
dot_size_dict = Dict()
for n in getleaflist(newt)
    dot_size_dict[n] = 10
    if seqname2chimeric[n.name]
        seqname2color[n] = "red"
    else
        seqname2color[n] = "blue"
    end
end
compose_dict = Dict()
for n in getleaflist(newt)
    compose_dict[n] = (x, y) -> pie_chart(x, y, [1], colors = [seqname2color[n.name]], size = 0.011)
end
tree_plot = tree_draw(newt, draw_labels = false, dot_color_dict = seqname2color, dot_size_dict = dot_size_dict, horizontal = true);
# add a legend
tree_plot = compose(tree_plot,
                (context(), circle(1, 1.00, .01), fill("blue")),
                (context(), circle(1, 0.97, .01), fill("red")),
                (context(), Compose.text(1.02, 1.006,"non-chimeric")),
                (context(), Compose.text(1.02, 0.976,"chimeric")))
tree_plot |> SVG(joinpath(trees_dir, "IML3695_IgM_25cycle_clone_id=$(clone_id)_withchimeras.svg"))
#end

┌ Info: Running `/usr/local/bin/mafft --thread 16 /tmp/jl_fyBD7Z/sequences.fasta`
└ @ Main /home/mchernys/ben/chimera_detection/chimera_detection/src/utils.jl:81
Ignored unknown character X (seen 14 times)


false

In [ ]:
# for IML3695 post-vax
# clones of interest: 21689, 1392, 16149, 19520, 20148, 22645, 2633, 6544, 9332
# for IML3694 post-vax
# clones of interest: 8354, 5329, 5025, 50, 19315, 1425